In [ ]:
#Source: Instruments & Data Tools
#Inspired from the Vector Quantization Example

from sklearn.cluster import KMeans, kmeans_plusplus
from sklearn.preprocessing import MinMaxScaler
from skimage.io import imread, imsave
from skimage.segmentation import slic
from skimage.color import rgb2gray, label2rgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from util import transform_image_to_coordinates, transform_coordinates_to_image

In [ ]:
# Read the data as greyscale 
#image = imread('./data/input/a4ca9877-c77b-4a8e-9d7a-aa621a7736fe.webp')
image = imread('./data/input/carpet.webp')
plt.imshow(image)

In [ ]:
X = transform_image_to_coordinates(image)*[1.0,1.0,1.0,1.0,1.0]
X[67845:67850]

In [ ]:
# use slic to segment image; uses K-Means under the hood
lables = slic(
    image,
    enforce_connectivity=False,
    n_segments=200,
    compactness=50
)
plt.imshow(label2rgb(lables, image))

In [ ]:
group_lable, group_index, pixel_count  = np.unique(lables, return_inverse=True, return_counts=True)
group_lable

In [ ]:
pixel_sum = np.zeros([*group_lable.shape,3])
np.add.at(
    pixel_sum,
    group_index,
    image.reshape(-1,3)
)
# pixel_count = np.zeros(group_lable.shape)
# np.add.at(
#     pixel_count,
#     group_index,
#     1
# )
# pixel_average = pixel_sum / pixel_count.reshape(-1,1)
pixel_average = np.round(pixel_sum/pixel_count.reshape(-1,1)).astype("u1")

In [ ]:
fig,axs = plt.subplots(1,2)
fig.set_size_inches(10,10)
axs[0].imshow(image)
axs[1].imshow(pixel_average[group_index].reshape(image.shape))

In [ ]:

scaler = MinMaxScaler((0,1)).fit(X)
X      = scaler.transform(X)

clustering = KMeans(
    n_clusters = 100,
    n_init     = 5,
    init       = "random",
)
clustering.fit(X)

In [ ]:
xy_coordinates = X[:,:2]
labels         = clustering.labels_
color_pallet   = clustering.cluster_centers_[:,2:]

colors         = color_pallet[labels]
result_image   = np.concatenate([xy_coordinates,colors], axis=1)

result_image = scaler.inverse_transform(result_image)
result_image = transform_coordinates_to_image(result_image)

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(20,20)
ax.imshow(np.round(result_image))

In [ ]:
imsave("./data/output/out.png",result_image)